<a href="https://colab.research.google.com/github/s20033/MIW_projects/blob/main/Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><h1>Implementation of AlexNet</b></h1> <br>
Using Keras Libraray implement AlexNet and train it to recognize fine grained classes from CIFAR100 dataset.

#Library Import


In [ ]:
#import various libraries

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

#from keras.optimizers import adam
from keras.callbacks import Callback

from keras.utils import np_utils # To transfor labels in categorical
from keras.datasets import cifar100 # to load the dataset

from keras import backend as K
#K.set_image_dim_ordering('tf') # To tell Tensorflow the right order of dims
# since I am using google colab and want to upload or download files
from google.colab import files
import os
import time


In [ ]:
import tensorflow as tf
from keras.models import model_from_json
import keras

<h2>Preparing the Data </h2>

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

In [ ]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

In [ ]:
validation_images, validation_labels = train_images[:5000], train_labels[:5000]
train_images, train_labels = train_images[5000:], train_labels[5000:]

In [ ]:
print(train_images.shape)

(45000, 32, 32, 3)


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 45000
Test data size: 10000
Validation data size: 5000


#Resize image size from (32, 32, 3) to (227, 227, 3)

In [ ]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

In [ ]:
train_ds

<BatchDataset shapes: ((32, 227, 227, 3), (32, 1)), types: (tf.float32, tf.int64)>

<h2> AlexNet Implementation. (Fused)</h2>
This implementation is optimized for modern GPU Training

Creating Layers

In [ ]:
model = keras.models.Sequential([
                                 # Layer conv1
    #keras.layers.ZeroPadding2D(padding=(0, 0), data_format=None),
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    # layer maxpool1
    keras.layers.ZeroPadding2D(padding=(0, 0), data_format=None),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    # layer conv2
    keras.layers.ZeroPadding2D(padding=(2, 2), data_format=None),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu'),
    # layer maxpool2
    keras.layers.ZeroPadding2D(padding=(0, 0), data_format=None),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    #layer conv3
    keras.layers.ZeroPadding2D(padding=(1, 1), data_format=None),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu'),
   # layer conv4
    keras.layers.ZeroPadding2D(padding=(1, 1), data_format=None),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu'),
    # layer conv5
    keras.layers.ZeroPadding2D(padding=(1, 1), data_format=None),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu'),
    #layer maxpool5
    keras.layers.ZeroPadding2D(padding=(0, 0), data_format=None),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    # layer flatten 6
    keras.layers.Flatten(),
    # layer flatten dense 7
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1000, activation='softmax')
])

<h2> Defining Model Structure </h2>

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_8 (ZeroPaddin (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
zero_padding2d_9 (ZeroPaddin (None, 27, 27, 256)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 256)      

In [ ]:
history = model.fit(train_ds,
          epochs=1,
          validation_data=validation_ds,
          validation_freq=1)

In [ ]:
tf.keras.utils.plot_model(model, show_layer_names=False, show_shapes=True, show_dtype=True)

Saving Model for later (In JSON)

In [ ]:
# save model architecture in JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# save Model weight in Hdf5 
model.save('model.h5'.format(1))
model.save_weights("model.hdf5")

# Tensor Board (optional)
we’ll be utilizing TensorBoard for visualization and monitoring of our model. 

In [ ]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In order to use tenor Board, open Terminal in directory level and run following comand:<br>
[tensorboard --logdir logs]

# Loading Model and visualizing Model Accuracy

In [ ]:
model = keras.models.load_model('model.h5'.format(1))
loss, acc = model.evaluate(test_ds)
print('accuracy: {}'.format(acc*100))
print('loss: {}'.format(loss))

# result
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.title('model 1')
plt.show()